In [1]:
import pandas as pd
import numpy as np
# from glob import glob
# import re
# import nltk
# import plotly_express as px
# from nltk.corpus import stopwords
# from nltk import pos_tag
# import os
# from textparser import TextParser
# import random
# from numpy.linalg import norm
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation as LDA
# from sklearn.decomposition import PCA
# from scipy.linalg import norm, eigh
# from scipy.spatial.distance import pdist
# import scipy.cluster.hierarchy as sch
# import matplotlib.pyplot as plt

In [36]:
pd.set_option('max_colwidth', 115)

In [3]:
OHCO = ['movie_id','scene_id', 'sent_num', 'token_num']
TOKES = OHCO[:4]
SENTS = OHCO[:3]
SCENES = OHCO[:2]
MOVIES = OHCO[:1]

In [4]:
LIB = pd.read_csv('/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/LibFinal.csv')

In [5]:
def tokenize_collection(LIB):

    clip_pats = []
    scene_regex = r'^\s*(?:INT\.|EXT\.|INT\/EXT\.|I\/E\/|EST\.|MONTAGE|INT\:|EXT\:|INT|EXT)\s+.*$'

    movies = []
    for movie_id in LIB.index:

        # Announce
        print("Tokenizing", movie_id, LIB.loc[movie_id].movie_title)

        # Define vars
        chap_regex = scene_regex
        ohco_pats = [('scene', chap_regex, 'm')]
        src_file_path = LIB.loc[movie_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)

        # Define parameters
        text.verbose = False
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['movie_id'] = movie_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['movie_id'] + text.OHCO)

        # Add to list
        movies.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(movies).sort_index()

    # Clean up
    del(movies)
    del(text)
        
    print("Done")
        
    return CORPUS

CORPUS = tokenize_collection(LIB)

Tokenizing 0 aliens
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 1 americanpsycho
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 2 buffythevampireslayer
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 3 dragmetohell
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 4 eightleggedfreaks
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 5 finaldestination
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 6 freddyvs.jason
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 7 fridaythe13thpartviiijasontakesmanhattan
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 8 ghostship
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 9 gingersnaps
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 10 gremlins
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 11 grudgethe
line_str scene_str
Index(['scene_str'

In [6]:
CORPUS

pos_tuple  pos token_str term_str
movie_id scene_id sent_num token_num                                       
0        1        0        0            (Dark, NNP)  NNP      Dark     dark
                           1              (and, CC)   CC       and      and
                           2          (dormant, JJ)   JJ   dormant  dormant
                           3               (as, IN)   IN        as       as
                           4                (a, DT)   DT         a        a
...                                             ...  ...       ...      ...
36       111      0        10         (beyond., NN)   NN   beyond.   beyond
                  1        0            (FADE, NNP)  NNP      FADE     fade
                           1            (OUT:, NNP)  NNP      OUT:      out
                           2             (THE, NNP)  NNP       THE      the
                           3              (END, NN)   NN       END      end

[839344 rows x 4 columns]

## Extract some features for `LIB`

In [7]:
LIB['movie_len'] = CORPUS.groupby('movie_id').term_str.count()

In [8]:
LIB['n_scenes'] = CORPUS.reset_index()[['movie_id','scene_id']]\
    .drop_duplicates()\
    .groupby('movie_id').scene_id.count()

In [9]:
LIB = LIB.set_index('movie_id')

In [10]:
LIB

,movie_title,source_file_path,year,decade,movie_len,n_scenes
movie_id,,,,,,
0,aliens,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/aliens.txt,1985,1980,27022,189
1,americanpsycho,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/americanpsy...,2000,2000,20104,57
2,buffythevampireslayer,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/buffythevam...,1992,1990,22254,123
3,dragmetohell,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/dragmetohel...,2009,2000,17154,75
4,eightleggedfreaks,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/eightlegged...,2002,2000,17418,195
5,finaldestination,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/finaldestin...,2000,2000,23594,142
6,freddyvs.jason,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/freddyvs.ja...,2003,2000,24336,131
7,fridaythe13thpartviiijasontakesmanhattan,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/fridaythe13...,1989,1980,23407,196
8,ghostship,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/ghostship.txt,2002,2000,23529,260


# Average Movie Length in Tokens

In [11]:
LIB.movie_len.mean()

22645.297297297297

# Exract VOCAB

Extract a vocabulary from the CORPUS as a whole

## Handle Anomalies

NLTK's POS tagger is not perfect -- note the classification of punctuation as nouns, verbs, etc. We remove these from our corups.

In [12]:
CORPUS[CORPUS.term_str == '']

pos_tuple  pos token_str term_str
movie_id scene_id sent_num token_num                                   
1        18       0        16           (&, CC)   CC         &         
                  88       19           (&, CC)   CC         &         
         30       37       2            (&, CC)   CC         &         
         41       65       5            (&, CC)   CC         &         
         47       7        22           (&, CC)   CC         &         
...                                         ...  ...       ...      ...
34       44       26       1          (?!, VBD)  VBD        ?!         
         68       52       3           ('?, NN)   NN        '?         
36       27       17       3            (&, CC)   CC         &         
                  27       3            (&, CC)   CC         &         
         88       1        1            (&, CC)   CC         &         

[271 rows x 4 columns]

In [13]:
CORPUS[CORPUS.term_str == ''].token_str.value_counts()

&             99
+             21
'             18
...?          18
/             16
..            13
_              9
#              9
....           9
"              9
*              6
=              6
>*             4
..?            2
./             2
.?             2
.'             2
!!!            2
",             2
%              1
._             1
`..            1
,=             1
}.             1
>              1
..."           1
?!             1
:.             1
"...           1
!"             1
,.             1
**             1
,"             1
__________     1
'.             1
______         1
"..            1
.{             1
::             1
?...           1
'?             1
Name: token_str, dtype: int64

In [14]:
CORPUS = CORPUS[CORPUS.term_str != '']

In [15]:
CORPUS

pos_tuple  pos token_str term_str
movie_id scene_id sent_num token_num                                       
0        1        0        0            (Dark, NNP)  NNP      Dark     dark
                           1              (and, CC)   CC       and      and
                           2          (dormant, JJ)   JJ   dormant  dormant
                           3               (as, IN)   IN        as       as
                           4                (a, DT)   DT         a        a
...                                             ...  ...       ...      ...
36       111      0        10         (beyond., NN)   NN   beyond.   beyond
                  1        0            (FADE, NNP)  NNP      FADE     fade
                           1            (OUT:, NNP)  NNP      OUT:      out
                           2             (THE, NNP)  NNP       THE      the
                           3              (END, NN)   NN       END      end

[839073 rows x 4 columns]

In [16]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))

## Add Stopwords

We use NLTK's built in stopword list for English. Note that we can add and subtract from this list, or just create our own list and keep it in our data model.

In [17]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [18]:
# sw.sample(10)

In [19]:
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [20]:
VOCAB[VOCAB.stop == 1].sample(10)

,n,n_chars,p,i,max_pos,n_pos,cat_pos,stop
term_str,,,,,,,,
your,1785,4,0.002131,8.874202,PRP$,6,"{NN, PRP$, NNP, VBZ, CD, JJ}",1
out,4890,3,0.005838,7.420292,RP,13,"{NN, VBD, RP, NNP, PRP, VBZ, VB, IN, VBP, MD, RB, JJ, NNS}",1
won,7,3,0.000008,16.868556,VBD,3,"{VBD, NN, VBP}",1
ain,1,3,0.000001,19.675911,NN,1,{NN},1
am,207,2,0.000247,11.982424,VBP,5,"{NN, NNP, VB, VBP, RB}",1
themselves,60,10,0.000072,13.769020,PRP,5,"{NN, VBD, PRP, VB, VBP}",1
having,139,6,0.000166,12.556969,VBG,4,"{NN, NNP, VBG, VBD}",1
between,311,7,0.000371,11.395140,IN,5,"{NN, NNP, VB, VBP, IN}",1
by,1798,2,0.002147,8.863733,IN,10,"{NN, RP, NNP, VBZ, VB, VBP, IN, RB, JJ, NNS}",1


In [21]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

In [22]:
VOCAB.sample(10)

,n,n_chars,p,i,max_pos,n_pos,cat_pos,stop,stem_porter,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,
healthier,2,9,0.000002,18.675911,JJR,2,"{RBR, JJR}",0,healthier,healthier,healthy
rattling,19,8,0.000023,15.427983,VBG,5,"{NN, NNP, VB, VBG, JJ}",0,rattl,rattl,rattl
sol,7,3,0.000008,16.868556,NNP,1,{NNP},0,sol,sol,sol
closeness,1,9,0.000001,19.675911,NN,1,{NN},0,close,close,clos
vulnerability,1,13,0.000001,19.675911,NN,1,{NN},0,vulner,vulner,vuln
invincibility,1,13,0.000001,19.675911,NN,1,{NN},0,invinc,invinc,invinc
orizzey,1,7,0.000001,19.675911,IN,1,{IN},0,orizzey,orizzey,orizzey
yate,1,4,0.000001,19.675911,NN,1,{NN},0,yate,yate,yat
intimacy,2,8,0.000002,18.675911,NN,1,{NN},0,intimaci,intimaci,intim


In [23]:
VOCAB

,n,n_chars,p,i,max_pos,n_pos,cat_pos,stop,stem_porter,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,
0,32,1,0.000038,14.675911,CD,3,"{CD, NN, VBP}",0,0,0,0
000,1,3,0.000001,19.675911,CD,1,{CD},0,000,000,000
0000,1,4,0.000001,19.675911,CD,1,{CD},0,0000,0000,0000
000i12i12,1,9,0.000001,19.675911,CD,1,{CD},0,000i12i12,000i12i12,000i12i12
003559,1,6,0.000001,19.675911,CD,1,{CD},0,003559,003559,003559
...,...,...,...,...,...,...,...,...,...,...,...
zulu,1,4,0.000001,19.675911,CD,1,{CD},0,zulu,zulu,zulu
zurbarans,1,9,0.000001,19.675911,NNP,1,{NNP},0,zurbaran,zurbaran,zurb
zydeco,2,6,0.000002,18.675911,NN,1,{NN},0,zydeco,zydeco,zydeco


## DOC table

In [24]:
DOC = LIB.drop(columns = ['source_file_path', 'decade', 'movie_len', 'n_scenes'])
DOC['title'] = LIB['movie_title'] + ', ' + LIB['year'].astype(str)
DOC

,movie_title,year,title
movie_id,,,
0,aliens,1985,"aliens, 1985"
1,americanpsycho,2000,"americanpsycho, 2000"
2,buffythevampireslayer,1992,"buffythevampireslayer, 1992"
3,dragmetohell,2009,"dragmetohell, 2009"
4,eightleggedfreaks,2002,"eightleggedfreaks, 2002"
5,finaldestination,2000,"finaldestination, 2000"
6,freddyvs.jason,2003,"freddyvs.jason, 2003"
7,fridaythe13thpartviiijasontakesmanhattan,1989,"fridaythe13thpartviiijasontakesmanhattan, 1989"
8,ghostship,2002,"ghostship, 2002"


# Show LIB, CORPUS, VOCAB, and DOC tables

In [25]:
LIB

,movie_title,source_file_path,year,decade,movie_len,n_scenes
movie_id,,,,,,
0,aliens,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/aliens.txt,1985,1980,27022,189
1,americanpsycho,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/americanpsy...,2000,2000,20104,57
2,buffythevampireslayer,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/buffythevam...,1992,1990,22254,123
3,dragmetohell,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/dragmetohel...,2009,2000,17154,75
4,eightleggedfreaks,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/eightlegged...,2002,2000,17418,195
5,finaldestination,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/finaldestin...,2000,2000,23594,142
6,freddyvs.jason,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/freddyvs.ja...,2003,2000,24336,131
7,fridaythe13thpartviiijasontakesmanhattan,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/fridaythe13...,1989,1980,23407,196
8,ghostship,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/HorrorSeen/ghostship.txt,2002,2000,23529,260


In [26]:
CORPUS

pos_tuple  pos token_str term_str
movie_id scene_id sent_num token_num                                       
0        1        0        0            (Dark, NNP)  NNP      Dark     dark
                           1              (and, CC)   CC       and      and
                           2          (dormant, JJ)   JJ   dormant  dormant
                           3               (as, IN)   IN        as       as
                           4                (a, DT)   DT         a        a
...                                             ...  ...       ...      ...
36       111      0        10         (beyond., NN)   NN   beyond.   beyond
                  1        0            (FADE, NNP)  NNP      FADE     fade
                           1            (OUT:, NNP)  NNP      OUT:      out
                           2             (THE, NNP)  NNP       THE      the
                           3              (END, NN)   NN       END      end

[839073 rows x 4 columns]

In [27]:
VOCAB

,n,n_chars,p,i,max_pos,n_pos,cat_pos,stop,stem_porter,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,
0,32,1,0.000038,14.675911,CD,3,"{CD, NN, VBP}",0,0,0,0
000,1,3,0.000001,19.675911,CD,1,{CD},0,000,000,000
0000,1,4,0.000001,19.675911,CD,1,{CD},0,0000,0000,0000
000i12i12,1,9,0.000001,19.675911,CD,1,{CD},0,000i12i12,000i12i12,000i12i12
003559,1,6,0.000001,19.675911,CD,1,{CD},0,003559,003559,003559
...,...,...,...,...,...,...,...,...,...,...,...
zulu,1,4,0.000001,19.675911,CD,1,{CD},0,zulu,zulu,zulu
zurbarans,1,9,0.000001,19.675911,NNP,1,{NNP},0,zurbaran,zurbaran,zurb
zydeco,2,6,0.000002,18.675911,NN,1,{NN},0,zydeco,zydeco,zydeco


In [28]:
DOC

,movie_title,year,title
movie_id,,,
0,aliens,1985,"aliens, 1985"
1,americanpsycho,2000,"americanpsycho, 2000"
2,buffythevampireslayer,1992,"buffythevampireslayer, 1992"
3,dragmetohell,2009,"dragmetohell, 2009"
4,eightleggedfreaks,2002,"eightleggedfreaks, 2002"
5,finaldestination,2000,"finaldestination, 2000"
6,freddyvs.jason,2003,"freddyvs.jason, 2003"
7,fridaythe13thpartviiijasontakesmanhattan,1989,"fridaythe13thpartviiijasontakesmanhattan, 1989"
8,ghostship,2002,"ghostship, 2002"


# Convert tables to CSVs

In [29]:
LIB.to_csv('/Users/theothormann/Desktop/Data Science/Spring/DS5001/FinalData/LIB.csv')

In [30]:
CORPUS.to_csv('/Users/theothormann/Desktop/Data Science/Spring/DS5001/FinalData/CORPUS.csv')

In [31]:
VOCAB.to_csv('/Users/theothormann/Desktop/Data Science/Spring/DS5001/FinalData/VOCAB_noTForDF.csv')

In [32]:
DOC.to_csv('/Users/theothormann/Desktop/Data Science/Spring/DS5001/FinalData/DOC.csv')